Vamos a listar las "jotas", es decir, el índice de las coordenadas totales, siguiendo este orden: de la esquina inferior [1,1], ese será j=1, luego j=2 corresponderá a [1,2], j=64 será [1,64] y luego j=65 será [2,1]. Recuerda que esto está ordenado como MATRICES, entonces j=5, [1,5]  es el lugar coordenada x=5 y y=1. Respeta el orden y todo saldrá bien.

In [1]:
ConjuntoDeCoordenadasTotal=Array[]
LasXNetas=Array[]
CasiXNetas=Array[]

0-element Array{Array{T,N},1}

Por cierto, estoy SEGURO que esta NO es la forma más inteligente de hacerlo, pero funciona. Ya después optimizamos. 
Primero, pongamos una lista de todas las coordenadas de los electrodos que nos interesa. La podemos leer de un archivo o crear *in-situ*.

In [2]:
for j=1:64,k=1:64
    push!(ConjuntoDeCoordenadasTotal,[j,k])
end

for j=1:24, k=1:24
    push!(LasXNetas,[j+5,k+23])
    push!(CasiXNetas,[j,k])
end

In [3]:
size(LasXNetas)

(576,)

In [23]:
BceroDura=readdlm("BceroDura-r-5.0.dat");
BceroSuave=readdlm("BceroSuave-r-5.0.dat")
btildedura=readdlm("BtildeDuraCero_radio_5.0.dat")
btildesuave=readdlm("BtildeSuaveCero_radio_5.0.dat");

In [24]:
BceroSuave[64,64]

116.65587463286872

In [ ]:
#BceroSuave[64,64]=393.105 #Convergencia, hay que checar el notebook anterior y ver porque pasa eso...

In [ ]:
#BceroDura[64,64]=137.089 #Correccion por convergencia, hay que checar el notebook anterior y ver porque pasa eso...

Dependiendo del cpu los procesos. En mi casa añado 5, en el labo añado 11.

In [8]:
addprocs(5)

5-element Array{Int64,1}:
 2
 3
 4
 5
 6

In [25]:
nprocs()

6

In [10]:
@everywhere function sumaalgunasB(LasB::Array, lasX::Array,rango)
    tantas=length(rango)
    tamanho=length(lasX)
    result=zeros(tamanho,tamanho)
    xconstante=[64,64]
    for j=1:tamanho
     xj=lasX[j]
         for k=1:j
            xk=lasX[k]
            aux=0
            for l in rango
                xl=lasX[l]
                aux += LasB[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    return result
    
end

@everywhere function sumaalgunasByBtilde(LasB::Array, LasBtilde::Array, lasX::Array,rango)
    tantas=length(rango)
    tamanho=length(lasX)
    result=zeros(tamanho,tamanho)
    xconstante=[64,64]
    for j=1:tamanho
     xj=lasX[j]
        for k=1:tamanho
            xk=lasX[k]
            aux=0
            for l in rango
                xl=lasX[l]
                aux += LasBtilde[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    return result
    
end


In [11]:
@time sumaalgunasB(BceroSuave, ConjuntoDeCoordenadasTotal, 1:2);

 45.976851 seconds (377.84 M allocations: 13.765 GB, 3.15% gc time)


In [12]:
@time sumaalgunasByBtilde(BceroSuave,btildedura,ConjuntoDeCoordenadasTotal,1:2)

 93.382037 seconds (754.99 M allocations: 27.376 GB, 3.18% gc time)


4096x4096 Array{Float64,2}:
 231.994   231.994   231.994   231.994   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 231.994   231.994   231.994   231.994      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 226.953   226.953   226.953   226.953      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 217.369   217.369   217.369   217.369      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 203.684   203.684   203.684   203.684      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 186.163   186.163   186.163   186.163   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 166.767   166.767   166.767   166.767      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 148.978   148.978   148.978   148.978      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 134.105   134.105   134.105   134.105      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 121.595   121.595   121.595   121.595      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 110.997   110.997   110.997   110.997   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 101.949   101.949   101.949   101.949      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  94.1634   94.1634   94.1634   94.1

In [13]:
@everywhere function divideArray(lasX::Array)
    #el proceso myid()==nprocs() va a ser el director de orquesta
    procesos=nprocs()-1
    largo=length(lasX)
    idx=myid()
    if idx==nprocs()
        return 1:0
    else
        splits=[round(Int, s) for s in linspace(0,largo,procesos+1)]
        return splits[idx]+1:splits[idx+1]
    end
end

In [14]:
@everywhere function obtenKParcial(LasB,lasX)
    rangito=divideArray(lasX)
    @show (rangito)
    result= sumaalgunasB(LasB, lasX,rangito)
    return result
end

@everywhere function obtenKtildeParcial(LasB,LasBTilde, lasX)
    rangito=divideArray(lasX)
    @show (rangito)
    result= sumaalgunasByBtilde(LasB, LasBTilde, lasX,rangito)
    return result
end


In [15]:
LasXchiquitas=LasXNetas[1:25];
@time  laKParcial=obtenKParcial(BceroDura,LasXchiquitas);

rangito = 1:102
 35.065591 seconds (294.96 M allocations: 10.788 GB, 3.43% gc time)


512x512 Array{Float64,2}:
      3.01458e5       0.0        …      0.0            0.0      0.0
      3.187e5         3.37529e5         0.0            0.0      0.0
      3.33292e5       3.53665e5         0.0            0.0      0.0
      3.44626e5       3.66413e5         0.0            0.0      0.0
      3.52153e5       3.75155e5         0.0            0.0      0.0
      3.55372e5       3.79333e5  …      0.0            0.0      0.0
      3.54679e5       3.79288e5         0.0            0.0      0.0
      3.51053e5       3.75961e5         0.0            0.0      0.0
      3.45315e5  370171.0               0.0            0.0      0.0
      3.38145e5  362625.0               0.0            0.0      0.0
 330090.0             3.53926e5  …      0.0            0.0      0.0
      3.2156e5        3.44563e5         0.0            0.0      0.0
      3.12835e5       3.34903e5         0.0            0.0      0.0
      ⋮                          ⋱                     ⋮           
  87225.9         9149

In [16]:

@time  laKParcial=obtenKtildeParcial(BceroDura, btildesuave,LasXchiquitas)

rangito = 1:102
 71.026153 seconds (588.51 M allocations: 21.522 GB, 3.54% gc time)


512x512 Array{Float64,2}:
 1761.77   1890.86   1987.9    …  151.116  141.206  131.541  122.155
 1861.08   2000.81   2108.97      155.473  145.228  135.248  125.565
 1943.22   2092.91   2212.33      160.042  149.446  139.134  129.139
 2004.45   2163.01   2293.43      164.823  153.858  143.199  132.877
 2041.27   2207.29   2348.06      169.815  158.465  147.443  136.779
 2050.48   2222.2    2372.33   …  175.018  163.265  151.864  140.845
 2035.08   2210.04   2367.74      180.443  168.27   156.475  145.084
 2001.81   2177.42   2339.87      186.114  173.502  161.293  149.514
 1955.79   2130.02   2294.34      192.051  178.98   166.338  154.152
 1900.85   2072.22   2236.18      198.278  184.723  171.626  159.013
 1839.91   2007.33   2169.29   …  204.813  190.75   177.176  164.114
 1775.17   1937.89   2096.64      211.678  197.082  183.005  169.471
 1708.33   1865.83   2020.52      218.895  203.736  189.131  175.101
    ⋮                          ⋱                      ⋮             
  329.48

In [26]:
function sacalaKenTrozos(LasB::Array, lasX::Array)
    largo=length(lasX)
    cachos=nprocs()
    aux=SharedArray(Float64,(largo,largo,cachos))
     @sync begin
        for p in procs()
            @async  aux[:,:,p]=remotecall_fetch(p, obtenKParcial, LasB, lasX)
        end
        end #cierra el syncbegin
    result=zeros(largo,largo)
    result=sum(aux,3)
    return result
    #return result
end

function KtildeenTrozos(LasB::Array, LasBtilde::Array, lasX::Array)
    largo=length(lasX)
    cachos=nprocs()
    aux=SharedArray(Float64,(largo,largo,cachos))
     @sync begin
        for p in procs()
            @async  aux[:,:,p]=remotecall_fetch(p, obtenKtildeParcial, LasB, LasBtilde, lasX)
        end
        end #cierra el syncbegin
    result=zeros(largo,largo)
    result=sum(aux,3)
    return result
    #return result
end

KtildeenTrozos (generic function with 1 method)

In [27]:
@time K3R7=sacalaKenTrozos(BceroDura,LasXNetas);

rangito = 1:115
	From worker 2:	rangito = 116:230
	From worker 3:	rangito = 231:346
	From worker 4:	rangito = 347:461
	From worker 5:	rangito = 462:576
	From worker 6:	rangito = 1:0
 68.638064 seconds (420.62 M allocations: 15.402 GB, 3.30% gc time)


In [28]:
@time KTilde3R7=KtildeenTrozos(BceroDura,btildedura, LasXNetas);

rangito = 1:115
	From worker 2:	rangito = 116:230
	From worker 3:	rangito = 231:346
	From worker 4:	rangito = 347:461
	From worker 5:	rangito = 462:576
	From worker 6:	rangito = 1:0
209.953541 seconds (839.75 M allocations: 30.730 GB, 1.82% gc time)


In [29]:
longus=size(K3R7,1)
K3R7=reshape(K3R7,longus,longus)
KTilde3R7=reshape(KTilde3R7,longus,longus)
K3R7=K3R7+transpose(K3R7)

for j=1:longus
    K3R7[j,j]=K3R7[j,j]/2
end
writedlm("KDura3R7-radio_5.dat",K3R7)
writedlm("KTildeDura3R7-radio_5.dat",KTilde3R7)



In [ ]:
@everywhere function obtenerKernelParalell(LasB::Array, lasX::Array, jmax::Int)
    result=SharedArray(Float64,(jmax,jmax))
    @sync @parallel for j=1:jmax
      xj=lasX[j]
         for k=1:jmax
            xk=lasX[k]
            aux=0
          for l=1:jmax
                xl=lasX[l]
                aux = LasB[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    result=Array(result)
    return result
end

@everywhere function obtenerKtildeParalell(LasB::Array,LasBTilde::Array, lasX::Array, jmax::Int)
    result=SharedArray(Float64,(jmax,jmax))
    @sync @parallel for j=1:jmax
        xj=lasX[j]
         for k=1:jmax
            xk=lasX[k]
         for l=1:jmax
                xl=lasX[l]
                result[j,k]=200
                unax=xk-xl+xconstante
                unay=xj-xl+xconstante
                if(LasBTilde[unax...]!=0.0)
                    println("hola tu")
                    
                #result[j,k]+= LasB[(xj-xl+xconstante)...]*LasBTilde[(xk-xl+xconstante)...]
                end
        end
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end 
    return result
end
    